In [1]:
import gpflow
import numpy as np
import math
import tensorflow as tf


In [2]:
import get_history as gh

In [3]:
import feather
import pandas as pd

In [4]:
def gather_cols(params, indices, name=None):
    """Gather columns of a 2D tensor.

    Args:
        params: A 2D tensor.
        indices: A 1D tensor. Must be one of the following types: ``int32``, ``int64``.
        name: A name for the operation (optional).

    Returns:
        A 2D Tensor. Has the same type as ``params``.
    """
    with tf.op_scope([params, indices], name, "gather_cols") as scope:
        # Check input
        params = tf.convert_to_tensor(params, name="params")
        indices = tf.convert_to_tensor(indices, name="indices")
        try:
            params.get_shape().assert_has_rank(2)
        except ValueError:
            raise ValueError('\'params\' must be 2D.')
        try:
            indices.get_shape().assert_has_rank(1)
        except ValueError:
            raise ValueError('\'params\' must be 1D.')

        # Define op
        p_shape = tf.shape(params)
        p_flat = tf.reshape(params, [-1])
        i_flat = tf.reshape(tf.reshape(tf.range(0, p_shape[0]) * p_shape[1],
                                       [-1, 1]) + indices, [-1])
        return tf.reshape(tf.gather(p_flat, i_flat),
                          [p_shape[0], -1])

In [67]:
np.eye(6)+.1*np.ones((6,6))

array([[1.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 1.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 1.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 1.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 1.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 1.1]])

In [68]:



class CustomKernel(gpflow.kernels.Kernel):
    def __init__(self,input_dim, mysession=None,rhos=None,select_users=None,
                 active_dims=None, ARD=None, name=None):
        super().__init__(input_dim)

        self.sigma_u = tf.constant(np.array([[1.0,0.1],[0.1,1.0]]))
        
        self.sigma_theta = tf.constant(np.array([[1.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 1.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 1.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 1.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 1.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 1.1]]))
        #print(self.sigma_theta)
        #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                  #dtype=gpflow.settings.float_type)
        
        self.sigma_v =  tf.constant(np.array([[10.0,0.0],[0.0,10.0]]))
        #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                  #dtype=gpflow.settings.float_type)
        self.mysession=mysession
        
        self.rhos = tf.constant(rhos)
        self.select_users = tf.constant(select_users)
        ##will it freak out if there is no parameter?
        
    
    @gpflow.params_as_tensors
    def rbf_custom(self, X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return tf.exp(-tf.divide(tf.square(tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
        return tf.constant(1.0,dtype=tf.float64)
    #tf.exp(-tf.subtract(X,X2) / float(2.2))
    
    @gpflow.params_as_tensors
    def get_all_sigmas(self,sigma_v,g_one,g_two,day_one,day_two):
    #sigmax = [[] for i in range(100)]
    #tf.placeholder(tf.float64, shape=(100, 2,2))
    
            
        #print('eff')
        #print(effects.shape)
        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        


        #print('eff')
        #print(effects.shape)
        #effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
    
    
    
        
        
        get_g = g_one
        get_g_two = g_two
        to_return = [[] for i in range(100)]
        for i in range(100):
            temp = [[] for i in range(100)]
            for j in range( 100):
                rho = self.rbf_custom(day_one[i], X2=day_two[j])
            #print(sess.run(rho))
                val = rho[0]
        #tf.scalar_mul(val,sigma_v)
            #print(val)
                this_sigma = tf.scalar_mul(val,sigma_v)
            #print(this_sigma)
                effects = tf.tensordot(get_g[i],this_sigma,axes=[[0],[1]])
                effects = tf.tensordot(effects,get_g_two[j],axes = [[0],[0]])
                temp[j]=effects
            to_return[i]=temp
        
        return to_return
    
    @gpflow.params_as_tensors
    def K(self, X, X2=None):
 

      
        
        f_one = gather_cols(X, [0,1,2,3,4,5], name=None)
        g_one = gather_cols(X, [6,7], name=None)
        user_id_one = gather_cols(X, [8], name=None)
        day_one = gather_cols(X, [9], name=None)



        #print(f_one.get_shape())
        
        if not X2 is None:
            print('called')
            f_two = gather_cols(X2, [0,1,2,3,4,5], name=None)
            g_two = gather_cols(X2, [6,7], name=None)  
            user_id_two = gather_cols(X2, [8], name=None)
            day_two = gather_cols(X2, [9], name=None)
            
         
            
        else:
            #print('called')
            user_id_two = X[-2]
            day_two = day_one
            f_two = f_one
            g_two=g_one
            
            
        
        
        #rho_term = self.rbf_custom(day_one,X2=day_two)
        #print('rho')
        #print(rho_term.get_shape())
        
        baselines = tf.tensordot(tf.transpose(f_one),self.sigma_theta,axes=[[0],[1]])
        #print('baselines')
        #print(baselines.get_shape())
        baselines = tf.tensordot(baselines,tf.transpose(f_two),axes=[[1],[0]])
        
        
        #rhos = tf.constant(np.array([[self.rbf_custom(day_one[i], X2=day_two[j])[0]] \
                                     #for i, j in zip([i for i in range(100)], [j for j in range(100)])]))
        
        
        #effects = self.get_all_sigmas(self.sigma_v,g_one,g_two,day_one,day_two)
            #print('look at shape')t
            #print(baselines)
        #print('baselines')
        #print(baselines.get_shape())
        
        #effect_term = tf.tensordot(rho_term,self.sigma_v,axes=[[0],[0]])

        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        #effect_term = tf.add(user_term,effect_term)
       #print('eff')
        #print(effect_term.shape)
        
        #print('eff')
        #print(effects.shape)
        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        


        #print('eff')
        #print(effects.shape)
        
        ##important
        effects = tf.tensordot(tf.transpose(g_one),self.sigma_u,axes=[[0],[1]])
       
        
        effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
        effects_one = tf.multiply(effects,self.select_users)
        
        
        
        effects = tf.tensordot(tf.transpose(g_one),self.sigma_v,axes=[[0],[1]])
       
        
        effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
        effects_two = tf.multiply(effects,self.rhos)
        
        effects = tf.add(effects_one,effects_two)
   
        
        #print('eff')
        #print(effects.shape)
        result = tf.add(baselines,effects)
        #effect_term = tf.add(user_term,effect_term)
        print('r')
        print(result.shape)
        
        
        return result
       
        
        #self.variance * tf.minimum(X, tf.transpose(X2))


In [69]:
##need to reformat history
def put_x_together(h):
    X = []
    
    for i in range(len(h.states)):
        state_vector = h.states[i]
        state_vector.append(float(h.users[i]))
        state_vector.append(float(h.day_function[i]))
        X.append(state_vector)
    return X


In [70]:
history = gh.get_history(8,100,5)

In [71]:
X = put_x_together(history)

In [72]:
X = np.array(X)

In [73]:
users = np.array([[float(X[i][8]==X[j][8]) for j in range(100)] for i in range(100)])

In [74]:
def rbf_custom_np( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return math.exp(-((X-X2)**2)/1.0)
        #tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
rdayone = [x[9] for x in X]
rdaytwo = rdayone
rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(100)] for i in range(100)])

In [75]:
y = np.array([[float(r)] for r in history.rewards])

In [76]:
sess = tf.Session()
k =  CustomKernel(7,mysession=sess,rhos=rhos,select_users=users)
m = gpflow.models.GPR(X,y, kern=k)

r
(100, 100)


In [77]:
term= m.kern.K(X,X2=X)

called


r
(100, 100)


In [78]:
trm = term.eval(session=sess)

In [79]:
trm

array([[0.28862542, 0.30055868, 0.19686557, ..., 0.13463545, 0.11918832,
        0.13980165],
       [0.30055868, 0.36896916, 0.24475435, ..., 0.13431461, 0.14495925,
        0.13709961],
       [0.19686557, 0.24475435, 0.46925072, ..., 0.13531481, 0.1514203 ,
        0.14094095],
       ...,
       [0.13463545, 0.13431461, 0.13531481, ..., 0.52475349, 0.1363179 ,
        0.31034796],
       [0.11918832, 0.14495925, 0.1514203 , ..., 0.1363179 , 0.39939361,
        0.13931746],
       [0.13980165, 0.13709961, 0.14094095, ..., 0.31034796, 0.13931746,
        0.51155875]])

In [51]:
trm2

array([[1.66353876, 1.69780878, 1.63547958, ..., 0.25203181, 0.22869123,
        0.21601148],
       [1.69780878, 1.73880637, 1.67145011, ..., 0.24288807, 0.21913171,
        0.20688737],
       [1.63547958, 1.67145011, 1.61775965, ..., 0.27670757, 0.25057367,
        0.23695399],
       ...,
       [0.25203181, 0.24288807, 0.27670757, ..., 1.35785202, 0.24851965,
        0.23455471],
       [0.22869123, 0.21913171, 0.25057367, ..., 0.24851965, 1.50455196,
        0.21207093],
       [0.21601148, 0.20688737, 0.23695399, ..., 0.23455471, 0.21207093,
        1.43131701]])

In [24]:
rhos

array([[1.00000000e+000, 1.00000000e+000, 1.00000000e+000, ...,
        1.65841048e-157, 1.92194773e-098, 3.68085585e-272],
       [1.00000000e+000, 1.00000000e+000, 1.00000000e+000, ...,
        1.65841048e-157, 1.92194773e-098, 3.68085585e-272],
       [1.00000000e+000, 1.00000000e+000, 1.00000000e+000, ...,
        1.65841048e-157, 1.92194773e-098, 3.68085585e-272],
       ...,
       [1.65841048e-157, 1.65841048e-157, 1.65841048e-157, ...,
        1.00000000e+000, 1.12535175e-007, 2.31952283e-016],
       [1.92194773e-098, 1.92194773e-098, 1.92194773e-098, ...,
        1.12535175e-007, 1.00000000e+000, 3.72007598e-044],
       [3.68085585e-272, 3.68085585e-272, 3.68085585e-272, ...,
        2.31952283e-016, 3.72007598e-044, 1.00000000e+000]])

In [25]:
gpflow.train.ScipyOptimizer().minimize(m,session=sess)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 651.681764
  Number of iterations: 36
  Number of functions evaluations: 37


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 651.681764
  Number of iterations: 36
  Number of functions evaluations: 37


In [26]:
m.as_pandas_table()

,class,prior,transform,trainable,shape,fixed_shape,value
GPR/likelihood/variance,Parameter,None,+ve,True,(),True,26715.57014297582


In [369]:
def obtain_kernel_and_sigma_u(all_xs,all_ys):
    k =  CustomKernel(all_xs.shape[1])
    m = gpflow.models.GPR(all_xs, all_ys, kern=k)
    gpflow.train.ScipyOptimizer().minimize(m,session=sess)
    #sigma_u = m.kern.sigma_u
    #.value
    cov = np.zeros((all_xs.shape[0],all_xs.shape[0]))
    term = m.kern.K(all_xs,X2=all_xs)
    print(m.as_pandas_table())
    #for i in range(all_xs.shape[0]):
        #for j in range(all_xs.shape[0]):
            #print(type(all_xs[i][0]))
           # term = m.kern.K(all_xs[i],all_xs[j])
            #term = tf.cast(term, tf.float32)
           # cov[i][j]=term
            #term.eval(session=sess)
            #term.eval(session=sess)
    return term.eval(session=sess)
#,sigma_u

In [370]:
cov = obtain_kernel_and_sigma_u(X,y)


(?, ?)
Tensor("GPR-e94f3f47-854/likelihood_1/strided_slice_1:0", dtype=float64)
Tensor("GPR-e94f3f47-854/likelihood_1/strided_slice_4:0", dtype=float64)
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 542.233190
  Number of iterations: 32
  Number of functions evaluations: 33


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 542.233190
  Number of iterations: 32
  Number of functions evaluations: 33


(100, 3)
called


[17.62061977 16.3121152  16.83757731 15.21358701 18.68006498  3.
 15.        ]
[17.62061977 16.3121152  16.83757731 15.21358701 18.68006498  3.
 15.        ]
                             class prior transform  trainable shape  \
GPR/likelihood/variance  Parameter  None       +ve       True    ()   

                         fixed_shape               value  
GPR/likelihood/variance         True  2684.0070684618827  


In [60]:
def get_RT(y,X):
    sigma_theta = np.array([[1.0,0.1,0.1],[0.1,1.0,0.1],\
                                    [0.1,0.1,1.0]])
    to_return = [y[i]-np.dot(np.transpose(X[i][0:3]),sigma_theta) for i in range(len(X))]
    return to_return

In [62]:
r_adjusted = get_RT(y,X)

In [80]:
def create_H():
    return np.array([1,0,0,0,0,0],[0,0,1,0,1,0]).T
    
    

In [63]:
def get_MT(new_x,sigma_theta,sigma_u,sigma_v,):
    
    

In [81]:
    sigma_theta = np.array([[1.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 1.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 1.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 1.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 1.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 1.1]])
    sigma_u = np.array([[1.0,0.1],[0.1,1.0]])
    
    sigma_v = np.array([[10.0,0.0],[0.0,10.0]])
    
    

In [118]:
#one = tf.constant([0,0])
#two = tf.constant([1,1])
inp= tf.constant([[1.0, 2, 3], [4, 5, 6.0], [7.0, 8, 9]])
tf.norm([inp,inp],ord=2)


<tf.Tensor 'norm_9/Squeeze:0' shape=() dtype=float32>

In [13]:
df = pd.DataFrame(X,columns=['f_1','f_2','f_3','g_1','g_2','user_id','user_day_id'])
path = 'X_data.feather'
feather.write_dataframe(df, path)
df_rewards = pd.DataFrame(y,columns=['reward'])
path = 'y_data.feather'
feather.write_dataframe(df_rewards, path)

In [16]:

gr = m.kern.K(X,X)

In [19]:
X.shape

(2, 6)

In [20]:
gr.eval()

1.0

In [123]:
p=[1,1]+[2,2,2]+[3,3,3]
pp=[p,p]

In [124]:
np.mean(np.array(pp),axis=0)

array([1., 1., 2., 2., 2., 3., 3., 3.])

In [125]:
pp

[[1, 1, 2, 2, 2, 3, 3, 3], [1, 1, 2, 2, 2, 3, 3, 3]]

In [32]:
    def rbf_custom( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return tf.exp(tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
        return tf.constant(1.0,dtype=tf.float64)

In [519]:
tx = tf.constant(X)
day_two = gather_cols(tx, [6], name=None)
z= rbf_custom(day_two,day_two)

In [520]:
z

<tf.Tensor 'Exp_756:0' shape=(100, 1) dtype=float64>

In [521]:
def get_all_sigmas(sigma_v,g_one,g_two,day_one,day_two):
    #sigmax = [[] for i in range(100)]
    #tf.placeholder(tf.float64, shape=(100, 2,2))
    
            
        #print('eff')
        #print(effects.shape)
        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        


        #print('eff')
        #print(effects.shape)
        #effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
    
    get_g = g_one
    get_g_two = g_two
    to_return = [[] for i in range(get_g.get_shape()[0])]
    for i in range(get_g.get_shape()[0]):
        temp = [[] for i in range(get_g_two.get_shape()[0])]
        for j in range( get_g_two.get_shape()[0]):
            rho = rbf_custom(day_one[i], X2=day_two[j])
            #print(sess.run(rho))
            val = sess.run(rho)[0]
        #tf.scalar_mul(val,sigma_v)
            #print(val)
            this_sigma = tf.scalar_mul(val,sigma_v)
            #print(this_sigma)
            effects = tf.tensordot(get_g[i],this_sigma,axes=[[0],[1]])
            effects = tf.tensordot(effects,get_g_two[j],axes = [[0],[0]])
            temp[j]=effects
        to_return[i]=temp
        
    return to_return

In [465]:
(sess.run(z[0])[0])

1.0

In [28]:
        day_one = gather_cols(X, [6], name=None)
        f_one = gather_cols(X, [0,1,2], name=None)
        g_one = gather_cols(X, [3,4], name=None)

In [38]:
rbf_custom(day_one[0], X2=day_two[0])[0]

<tf.Tensor 'strided_slice_100004:0' shape=() dtype=float64>

In [30]:
g_two=g_one
day_two=day_one

In [49]:
ts = tf.stack([day_one,day_two],axis=1)

In [54]:
elems = ts

alternate = tf.map_fn(lambda x: rbf_custom(x[0], X2=x[1]), elems, dtype=(tf.tensor))

AttributeError: module 'tensorflow' has no attribute 'tensor'

In [72]:
rhos.shape

(100, 100)

In [433]:
sigma_theta =tf.constant(np.array([[100.0,10.0,10.0],[10.0,100.0,10.0],\
                                    [10.0,10.0,100.0]]))
sigma_v =  tf.constant(np.array([[100.0,0.0],[0.0,100.0]]))
sigma_u =  tf.constant(np.array([[100.0,10.0],[10.0,100.0]]))

In [199]:
user_id_one = X_one[-2]
day_one = X_one[-1]
f_one = X_one[0:3]
g_one = X_one[3:5]

user_id_two = X_two[-2]
day_two = X_two[-1]
f_two = X_two[0:3]
g_two = X_two[3:5]
        
        
rho_term = rbf_custom(day_one,X2=day_two).eval(session=sess) 
#print(sess.run(rho_term))
print('rho')
#print(tf.shape(f_one))
print(rho_term)
print(f_one.shape)
print(f_one)
#print(rho_term.get_shape()[1])
            #rho_term = tf.constant(np.array([1.0]))
       
baselines = tf.tensordot(tf.transpose(f_one),sigma_theta,axes=[[0],[0]])
print('baselines')
print(baselines.get_shape())
baselines = tf.tensordot(tf.transpose(baselines),f_two,axes=[[0],[0]])
            #print('look at shape')
            #print(baselines)
print('baselines')
print(baselines.get_shape())
print(baselines.eval(session=sess))
#if user_id_one==user_id_two:
#    user_term = self.sigma_u
#else:
#print('user')
#print('user_term')

            #print(self.sigma_v.shape)
            #print(rho_term.shape)
effect_term = tf.scalar_mul(rho_term,sigma_v)
#print('effect term')
#print(effect_term.shape)
#print('user term')
#print(user_term.shape)
user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
effect_term = tf.add(user_term,effect_term)
#print('eff')
#print(effect_term.shape)
#print('g')
#print(g_one.shape)
effects = tf.tensordot(tf.transpose(g_one),effect_term,axes=[[0],[1]])
print('eff')
print(effects.shape)
effects = tf.tensordot(tf.transpose(effects),g_two,axes = [[0],[0]])
print('eff')
print(effects.shape)
result = tf.add(baselines,effects)
print('res')
print(result.shape)
print(result.eval(session=sess))

rho
1.0
(3,)
Tensor("strided_slice_206:0", shape=(3,), dtype=float64)
baselines
(3,)
baselines
()
107826.87763336947
eff
(2,)
eff
()
res
()
169293.14642401805


In [222]:
X = np.vstack([np.random.randn(10,2) + [2,2],
               np.random.randn(10,2) + [-2,2],
               np.random.randn(10,2) + [2,-2]])

In [223]:
X.shape

(30, 2)

In [ ]:
 #.eval(session=sess) 
            #print(sess.run(rho_term))
            print('rho')
            #print(tf.shape(f_one))
            print(rho_term)
            print(f_one.shape)
            print(f_one)
            #print(rho_term.get_shape()[1])
            #rho_term = tf.constant(np.array([1.0]))
       
            baselines = tf.tensordot(tf.transpose(f_one),self.sigma_theta,axes=[[0],[0]])
            print('baselines')
            print(baselines.get_shape())
            baselines = tf.tensordot(tf.transpose(baselines),f_two,axes=[[0],[0]])
            #print('look at shape')
            #print(baselines)
            print('baselines')
            print(baselines.get_shape())
           
            #if user_id_one==user_id_two:
            #    user_term = self.sigma_u
            #else:
            #print('user')
            #print('user_term')

            #print(self.sigma_v.shape)
            #print(rho_term.shape)
            effect_term = tf.tensordot(rho_term,self.sigma_v,axes=[[0],[0]])
                #print('effect term')
            #print(effect_term.shape)
            #print('user term')
            #print(user_term.shape)
            user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
            effect_term = tf.add(user_term,effect_term)
            #print('eff')
            #print(effect_term.shape)
            #print('g')
            #print(g_one.shape)
            effects = tf.tensordot(tf.transpose(g_one),effect_term,axes=[[0],[1]])
            print('eff')
            print(effects.shape)
            effects = tf.tensordot(tf.transpose(effects),g_two,axes = [[0],[0]])
            print('eff')
            print(effects.shape)
            result = tf.add(baselines,effects)
            print('res')
            print(result.shape)
        